In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import numpy as np

# 1. WebMD reviews

In [3]:
df = pd.read_csv('csv-files/Kidney Stone Reviews - Master Sheet - WebMD.csv')
print(f"Successfully loaded {len(df)} rows of data")

Successfully loaded 1572 rows of data


In [4]:
PLOT_PARAMS = {
    'width': 400,
    'height': 200,
    'margin': dict(l=50, r=50, t=50, b=50),
    'title_font_size': 14
}

figfolder = "docs/"

In [5]:
for medicine in df['Medicine'].unique():
    medicine_data = df[df['Medicine'] == medicine]
    
    # Manually calculate the bins
    bins = [1.0, 2.0, 3.0, 4.0, 5.0, 5.1]  # 5.1 to include 5.0 in last bin
    counts, _ = np.histogram(medicine_data['Overall Rating'], bins=bins)
    
    # Create bar plot
    fig = go.Figure(data=[
        go.Bar(
            x=[1, 2, 3, 4, 5],  # Center bars on these x values
            y=counts,
            width=0.8,  # Bar width
            hovertemplate=(
                "Overall Rating: %{customdata}<br>" +
                "Count: %{y}" +
                "<extra></extra>"
            ),
            customdata=["1-1.9", "2-2.9", "3-3.9", "4-4.9", "5"]
        )
    ])

    fig.update_layout(
        title=f'Distribution of Ratings: {medicine}',
        xaxis_title='Rating',
        yaxis_title='Count',
        template='plotly_white',
        width=PLOT_PARAMS['width'],
        height=PLOT_PARAMS['height'],
        bargap=0.1,
        margin=PLOT_PARAMS['margin'],
        title_font_size=PLOT_PARAMS['title_font_size'],
        xaxis=dict(
            tickmode='array',
            tickvals=[1, 2, 3, 4, 5],
            ticktext=['1', '2', '3', '4', '5'],
            range=[0.5, 5.5]
        )
    )

    fig.show()

In [ ]:
all_data = []
for medicine in sorted(df['Medicine'].unique()):
    medicine_data = df[df['Medicine'] == medicine]
    bins = [1.0, 2.0, 3.0, 4.0, 5.0, 5.1]
    counts, _ = np.histogram(medicine_data['Overall Rating'], bins=bins)
    
    # Normalize the counts to percentages
    total_reviews = len(medicine_data)
    percentages = (counts / total_reviews) * 100
    
    ratings_df = pd.DataFrame({
        'Rating': [1, 2, 3, 4, 5],
        'Percentage': percentages,
        'Medicine': medicine,
        'RatingRange': ['[1,2)', '[2,3)', '[3,4)', '[4,5)', '5']  # Add rating ranges for hover
    })
    all_data.append(ratings_df)

plot_df = pd.concat(all_data)

fig = px.bar(plot_df, 
             x='Rating', 
             y='Percentage',
             color='Medicine',
             barmode='group',
             width=PLOT_PARAMS['width'] * 2.0,
             height=PLOT_PARAMS['height'] * 2.0)

fig.update_layout(
    title='Distribution of ratings by treatment (%)',
    xaxis_title='Rating',
    yaxis_title='Percentage of Reviews',
    template='plotly_white',
    bargap=0.15,
    bargroupgap=0.1,
    margin=PLOT_PARAMS['margin'],
    title_font_size=PLOT_PARAMS['title_font_size'],
    xaxis=dict(
        tickmode='array',
        tickvals=[1, 2, 3, 4, 5],
        ticktext=['[1,2)', '[2,3)', '[3,4)', '[4,5)', '5'],
        range=[0.5, 5.5]
    ),
    legend=dict(
        orientation='h',
        yanchor='top',
        y=-0.25,  # Move legend below x-axis title
        xanchor='center',
        x=0.5,
        title=None
    )
)

# Update the traces differently since we have grouped bars
for medicine_name in plot_df['Medicine'].unique():
    medicine_data = plot_df[plot_df['Medicine'] == medicine_name]
    fig.update_traces(
        customdata=medicine_data[['Medicine', 'RatingRange']].values,
        hovertemplate=(
            "%{customdata[0]}<br>" +
            "Rating: %{customdata[1]}<br>" +
            "Percentage: %{y:.1f}%" +
            "<extra></extra>"
        ),
        selector=dict(name=medicine_name)  # This ensures we update the correct trace
    )

fig.show()
figname = "webmd-treatment-distribution-rating"
fig.write_html(figfolder+figname+".html")
fig.write_image(figfolder+figname+".png", scale=4)

In [62]:
def calculate_stats(df, columns_to_analyze):

    # Calculate basic stats first
    summary = df.groupby('Medicine').agg(
        total_reviews=('Medicine', 'size'),
        **{
            f'{col}_positive': (col, lambda x: sum(
                1 for val in x if val == 1 or (isinstance(val, str) and val.strip())
            ))
            for col in columns_to_analyze
        },
        **{
            f'{col}_negative': (col, lambda x: sum(
                1 for val in x if val == 0  # Only count explicit 0s as negative
            ))
            for col in columns_to_analyze
        }
    ).reset_index()

    # Calculate percentages for each column
    for col in columns_to_analyze:
        summary[f'{col}_positive_pct'] = (summary[f'{col}_positive'] / summary['total_reviews'] * 100).round(1)
        summary[f'{col}_negative_pct'] = (summary[f'{col}_negative'] / summary['total_reviews'] * 100).round(1)
        summary[f'{col}_no_info_pct'] = (
            (summary['total_reviews'] - summary[f'{col}_positive'] - summary[f'{col}_negative']) 
            / summary['total_reviews'] * 100
        ).round(1)
    
    # Filter out excluded medicines
    medicines_to_exclude = ['Ashwagandha', 'Melatonin']
    summary = summary[~summary['Medicine'].isin(medicines_to_exclude)]
    
    # Create transposed dataframe
    transposed_data = {
        ' ': ['<b>Total Reviews</b>'] + 
            sum([[f'<b>{col}</b>', 
                 '    Yes', 
                 '    No', 
                 '    No information'] 
                for col in columns_to_analyze], []),
        **{row['Medicine']: [
            row['total_reviews']] + 
            sum([[
                '',  # Header row
                row[f'{col}_positive_pct'],
                row[f'{col}_negative_pct'],
                row[f'{col}_no_info_pct']
            ] for col in columns_to_analyze], [])
           for _, row in summary.iterrows()}
    }
    
    display_df = pd.DataFrame(transposed_data).set_index(' ')
    
    return display_df

columns_to_analyze = [
    'Helps overall with kidney stones',
    'Works as a prophylactic',
    'Side effects mentioned',
    'Asserts significant reduction in pain on net',
    'Mentions breaking of stones',
    'Mentions shrinking of the stones',
    'Mentions softening of stones',
    'Stone passed with no or almost no pain',
    'Promotes specific brand',
]

display_df = calculate_stats(df, columns_to_analyze)

display(display_df.style
        .format(lambda x: f'{x:,.0f}' if isinstance(x, (int, float)) and str(x).isdigit() 
                else (f'{x:.0f}%' if isinstance(x, float) and x.is_integer() 
                else f'{x:.1f}%' if isinstance(x, float) 
                else x)))

,Allopurinol Oral,Black Seed,Chanca Piedra,Flomax Oral,Garcinia,Hydrochlorothiazide,Potassium Citrate Oral
,,,,,,,
Total Reviews,14,77,91,22,892,20,24
Helps overall with kidney stones,,,,,,,
Yes,35.7%,0%,84.6%,4.5%,0%,30%,41.7%
No,7.1%,0%,3.3%,13.6%,0%,20%,4.2%
No information,57.1%,100%,12.1%,81.8%,100%,50%,54.2%
Works as a prophylactic,,,,,,,
Yes,35.7%,0%,13.2%,0%,0%,0%,37.5%
No,0%,0%,0%,0%,0%,0%,4.2%
No information,64.3%,100%,86.8%,100%,100%,100%,58.3%


# 2. Amazon reviews

In [92]:
dfa = pd.read_csv('csv-files/Kidney Stone Reviews - Master Sheet - Amazon.csv')
print(f"Successfully loaded {len(dfa)} rows of data")

Successfully loaded 1456 rows of data


In [93]:
def calculate_stats(df, columns_to_analyze):

    # Calculate basic stats first
    summary = df.groupby('Medicine').agg(
        total_reviews=('Medicine', 'size'),
        **{
            f'{col}_positive': (col, lambda x: sum(
                1 for val in x if val == 1 or (isinstance(val, str) and val.strip())
            ))
            for col in columns_to_analyze
        },
        **{
            f'{col}_negative': (col, lambda x: sum(
                1 for val in x if val == 0  # Only count explicit 0s as negative
            ))
            for col in columns_to_analyze
        }
    ).reset_index()

    # Calculate percentages for each column
    for col in columns_to_analyze:
        summary[f'{col}_positive_pct'] = (summary[f'{col}_positive'] / summary['total_reviews'] * 100).round(1)
        summary[f'{col}_negative_pct'] = (summary[f'{col}_negative'] / summary['total_reviews'] * 100).round(1)
        summary[f'{col}_no_info_pct'] = (
            (summary['total_reviews'] - summary[f'{col}_positive'] - summary[f'{col}_negative']) 
            / summary['total_reviews'] * 100
        ).round(1)
    
    # Filter out excluded medicines
    medicines_to_exclude = ['Chanca piedra']
    summary = summary[~summary['Medicine'].isin(medicines_to_exclude)]
    
    # Create transposed dataframe
    transposed_data = {
        ' ': ['<b>Total Reviews</b>'] + 
            sum([[f'<b>{col}</b>', 
                 '    Yes', 
                 '    No', 
                 '    No information'] 
                for col in columns_to_analyze], []),
        **{row['Medicine']: [
            row['total_reviews']] + 
            sum([[
                '',  # Header row
                row[f'{col}_positive_pct'],
                row[f'{col}_negative_pct'],
                row[f'{col}_no_info_pct']
            ] for col in columns_to_analyze], [])
           for _, row in summary.iterrows()}
    }
    
    display_df = pd.DataFrame(transposed_data).set_index(' ')
    
    return display_df

columns_to_analyze = [
    'Helps overall with kidney stones',
    'Works as a prophylactic',
    'Side effects mentioned',
    'Asserts significant reduction in pain on net',
    'Mentions breaking of stones',
    'Mentions shrinking of the stones',
    'Mentions softening of stones',
    'Stone passed with no or almost no pain',
    'Promotes specific brand',
]

display_df = calculate_stats(dfa, columns_to_analyze)

display(display_df.style
        .format(lambda x: f'{x:,.0f}' if isinstance(x, (int, float)) and str(x).isdigit() 
                else (f'{x:.0f}%' if isinstance(x, float) and x.is_integer() 
                else f'{x:.1f}%' if isinstance(x, float) 
                else x)))

,Phosfood,Potassium citrate,Rowatinex
,,,
Total Reviews,40,133,90
Helps overall with kidney stones,,,
Yes,75%,91%,90%
No,10%,0%,2.2%
No information,15%,9%,7.8%
Works as a prophylactic,,,
Yes,22.5%,66.9%,25.6%
No,5%,0%,1.1%
No information,72.5%,33.1%,73.3%


In [94]:
def calculate_stats_by_rating(df, columns_to_analyze, medicine_name, ratings, sources_to_exclude=None):
    # Create dictionary to store DataFrames for each rating
    rating_dfs = {}
    
    for rating in ratings:
        # Filter for specific medicine and rating
        filtered_df = df[(df['Medicine'] == medicine_name) & (df['Stars'] == rating)]
        
        # Exclude specified sources if any
        if sources_to_exclude:
            filtered_df = filtered_df[~filtered_df['Source'].isin(sources_to_exclude)]
        
        # Calculate stats for this rating
        summary = pd.DataFrame({
            'Medicine': [f'{medicine_name} ({rating}-star reviews)'],
            'total_reviews': [len(filtered_df)]
        })
        
        # Rest of the function remains the same
        for col in columns_to_analyze:
            summary[f'{col}_positive'] = [sum(
                1 for val in filtered_df[col] if val == 1 or (isinstance(val, str) and val.strip())
            )]
            summary[f'{col}_negative'] = [sum(
                1 for val in filtered_df[col] if val == 0
            )]
        
        for col in columns_to_analyze:
            total = summary['total_reviews'].iloc[0]
            if total > 0:
                summary[f'{col}_positive_pct'] = (summary[f'{col}_positive'] / total * 100).round(1)
                summary[f'{col}_negative_pct'] = (summary[f'{col}_negative'] / total * 100).round(1)
                summary[f'{col}_no_info_pct'] = (
                    (total - summary[f'{col}_positive'] - summary[f'{col}_negative']) 
                    / total * 100
                ).round(1)
            else:
                summary[f'{col}_positive_pct'] = 0
                summary[f'{col}_negative_pct'] = 0
                summary[f'{col}_no_info_pct'] = 0
        
        rating_dfs[rating] = summary
    
    transposed_data = {
        ' ': ['<b>Total Reviews</b>'] + 
            sum([[f'<b>{col}</b>', 
                 '    Yes', 
                 '    No', 
                 '    No information'] 
                for col in columns_to_analyze], [])
    }
    
    for rating in ratings:
        summary = rating_dfs[rating]
        transposed_data[summary['Medicine'].iloc[0]] = [
            summary['total_reviews'].iloc[0]] + sum([[
                '',  # Header row
                summary[f'{col}_positive_pct'].iloc[0],
                summary[f'{col}_negative_pct'].iloc[0],
                summary[f'{col}_no_info_pct'].iloc[0]
            ] for col in columns_to_analyze], [])
    
    display_df = pd.DataFrame(transposed_data).set_index(' ')
    
    return display_df

# Example usage:
# sources_to_exclude = ['Peruvian Naturals Chanca Piedra Capsules', 'NaturalisimoLife Chanca Piedra 1600 mg', 'EU Natural: "Stone Breaker" chanca piedra']
# sources_to_exclude = ['EU Natural: "Stone Breaker" chanca piedra']
sources_to_exclude = []
ratings_display_df = calculate_stats_by_rating(dfa, columns_to_analyze, 'Chanca piedra', [1, 5], sources_to_exclude)

display(ratings_display_df.style
        .format(lambda x: f'{x:,.0f}' if isinstance(x, (int, float)) and str(x).isdigit() 
                else (f'{x:.0f}%' if isinstance(x, float) and x.is_integer() 
                else f'{x:.1f}%' if isinstance(x, float) 
                else x)))

,Chanca piedra (1-star reviews),Chanca piedra (5-star reviews)
,,
Total Reviews,200,590
Helps overall with kidney stones,,
Yes,0.5%,86.3%
No,87.5%,0.3%
No information,12%,13.4%
Works as a prophylactic,,
Yes,0%,21%
No,1.5%,0.5%
No information,98.5%,78.5%
